# INFO

Este notebook se empleó como forma de automatizar la generación de SPARQL a partir de ciertos NLQ, las consultas de idearon manualmente y aquí se automatizó la generación de SPARQL, validándose manualmente las consultas y ejecutándose en el TripleStore

In [ ]:
%pip install requests openai pyparseit SPARQLWrapper

In [ ]:
n = 38

samples = [
    # NLQs
]

In [ ]:
import json
import time
import openai
import pyparseit
import SPARQLWrapper

client = openai.OpenAI(
            base_url="http://127.0.0.1:1234/v1/",
            api_key="none",
        )

model = "unsloth/qwen3-30b-a3b"

max_tokens = 8*1024

sparql = SPARQLWrapper.SPARQLWrapper(endpoint="http://192.168.20.166:8890/sparql")

In [ ]:
import SPARQLWrapper.SPARQLExceptions
from IPython.display import clear_output


prompt_template = """
Write a SPARQL query for this natural language question:

```
{nlq}
```
"""

results = []

for sample in samples:

    valid = False
    result = {}

    while not valid:
        
        clear_output(wait=True)
        
        print("NLQ:")
        print(sample)
        
        good_query = False
        
        prompt = prompt_template.replace("{nlq}", sample)

        response = client.chat.completions.create(
            model="none",
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            max_tokens=max_tokens,
        )
        
        text = response.choices[0].message.content

        snippets = pyparseit.parse_markdown_string(text)

        result["text"] = text
        #result["raw"] = response.model_dump()
        
        if len(snippets) >= 1:

            result["sparql"] = snippets[-1].content
                
            print("\n\nSPARQL:")
            print(result["sparql"])
            
            test_query = result["sparql"]
            
            if not "LIMIT" in test_query:
                test_query = test_query+"\nLIMIT 1000"
                
            try:

                sparql.setQuery(test_query)
                sparql.setReturnFormat(SPARQLWrapper.JSON)
                query_result = sparql.queryAndConvert()

                result["sparql_result"] = query_result
                
                good_query = True
                
                print("\n\nRESULT:")
                print(json.dumps(query_result, indent=4))
                
            except SPARQLWrapper.SPARQLExceptions.QueryBadFormed:
                
                good_query = False
                
                print("Bad query")
                
            except SPARQLWrapper.SPARQLExceptions.EndPointInternalError:
                
                good_query = False
                
                print("Endpoint error")
                
                

            
            time.sleep(1)
            
            decision = input("Valid? [y/n/s]")
            
            if good_query and decision == "y":
                
                with open(f"nlq/q{n}.nlq", "w+") as nlq_file:
                    nlq_file.write(sample)
                    
                with open(f"sparql/q{n}.sparql", "w+") as sparql_file:
                    sparql_file.write(result["sparql"])
                    
                with open(f"result/q{n}.json", "w+") as json_file:
                    json_file.write(json.dumps(query_result, indent=4))


                n += 1
                
                valid = True
                
            elif decision == "s":
                break
               
